# Парсинг новостных статей

In [ ]:
!pip install requests beautifulsoup4 selenium webdriver-manager google_colab_selenium


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.2 MB/s eta 0:00:00


In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import time
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import google_colab_selenium as gs
from concurrent.futures import ThreadPoolExecutor, as_completed

# Конфигурация источников
sources = [
    {
        'name': 'ria.ru',
        'news_url': 'https://ria.ru/lenta/',
        'article_selector': 'a.list-item__title, a[href*="/2025"]',
        'title_selectors': ['h1.article__second-title', 'div.article__title', 'h1', '[itemprop="headline"]'],
        'text_selectors': ['.article__text', '.article__block[data-type="text"] .article__text', 'div.article__body p', 'article p', 'div.article__body div', '.article__content p', '.article__content div'],
        'date_selectors': ['.article__info-date a', 'time', '[itemprop="datePublished"]', '[property="article:published_time"]'],
        'category_selectors': ['.article__tags-item', '[data-analytics-rubric]', 'a[href*="/world/"], a[href*="/politics/"]'],
        'dynamic': True
    },
    {
        'name': 'lenta.ru',
        'news_url': 'https://lenta.ru/',
        'article_selector': 'a[href*="/news/"]',
        'title_selectors': ['h1', '.topic-header__title', '.article__title'],
        'text_selectors': ['.topic-body__content p', '.article__body p', '.topic-body__text', '.article__text'],
        'date_selectors': ['time', '.publish-date', '[itemprop="datePublished"]', 'meta[property="article:published_time"]'],
        'category_selectors': ['.rubric', '.category', '[data-rubric]', '.topic-header__rubric'],
        'dynamic': True
    },
    {
        'name': 'tass.ru',
        'news_url': 'https://tass.ru/ekonomika',
        'article_selector': 'a.NewsCard_link__[data-testid], a.news-preview__link, a.card__link, a[href*="/ekonomika/"]',
        'title_selectors': ['h1.NewsCard_title__[data-testid]', 'h1.article__title', 'h1', '[itemprop="headline"]'],
        'text_selectors': ['div.TextBlock_wrapper__[data-testid] p', 'div.text-block p', 'article p', '.ArticleBody_wrapper__[data-testid] p', '.article__text p', '.news-text p', '.NewsCard_text__[data-testid]', '.article-content p'],
        'date_selectors': ['time.NewsCard_date__[data-testid]', 'span.datetime__[data-testid]', 'time', '[itemprop="datePublished"]'],
        'category_selectors': ['a.Tag_wrapper__[data-testid]', 'div.category a', '[data-category]', 'meta[name="category"]'],
        'dynamic': True
    },
    {
        'name': 'kommersant.ru',
        'news_url': 'https://www.kommersant.ru/lenta',
        'article_selector': 'a.uho__link, a.article-link, a[href*="/doc/"]',
        'title_selectors': ['h1.doc_header__name', 'h1.article__title', '[itemprop="headline"]'],
        'text_selectors': ['div.doc__text p', '.article__body p', 'article p', '.doc__text div'],
        'date_selectors': ['time.doc_header__publish_time', 'time', '[itemprop="datePublished"]'],
        'category_selectors': ['div.doc_header__rubric', 'a.category', '[data-rubric]'],
        'dynamic': True
    },
]

def setup_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    return gs.Chrome(options=options)

def extract_from_meta(soup, url):
    """Извлечение данных из мета-тегов и общих блоков"""
    data = {}
    data['title'] = (soup.find('title').text.strip() if soup.find('title') else
                     soup.find('meta', property='og:title')['content'] if soup.find('meta', property='og:title') else
                     'N/A')

    date_match = re.search(r'/(\d{4})(\d{2})(\d{2})/', url)
    if date_match:
        data['date'] = f"{date_match.group(1)}-{date_match.group(2)}-{date_match.group(3)}"
    else:
        date_meta = soup.find('meta', property='article:published_time')
        data['date'] = date_meta['content'] if date_meta else 'N/A'

    rubric_meta = soup.find('meta', attrs={'name': re.compile('analytics:rubric|category', re.I)})
    data['category'] = rubric_meta['content'] if rubric_meta else 'N/A'

    text_parts = []
    for block in soup.find_all(['div', 'article'], class_=['TextBlock_wrapper__[data-testid]', 'text-block', 'ArticleBody_wrapper__[data-testid]',
                                                          'article__block', 'news-full__text', 'topic-body__content', 'doc__text', 'general-material__body']):
        text_elem = block.find_all(['p', 'div'], class_=['article__text', 'news-full__content', 'topic-body__text', 'text-block', 'doc__text',
                                                        'TextBlock_wrapper__[data-testid]', 'general-material__text', 'NewsCard_text__[data-testid]', 'article-content'])
        for elem in text_elem:
            if elem.text.strip():
                text_parts.append(elem.text.strip())

    if not text_parts:
        desc_meta = soup.find('meta', property='og:description')
        text_parts = [desc_meta['content']] if desc_meta else []

    data['text'] = ' '.join(text_parts)
    data['url'] = url
    return data

def get_article_links(source, limit=50):
    """Получение ссылок на статьи с главной страницы"""
    links = []
    driver = setup_driver()
    wait = WebDriverWait(driver, 10)

    try:
        print(f"[{source['name']}] Загружаем главную страницу: {source['news_url']}")
        driver.get(source['news_url'])
        wait.until(EC.any_of(
            EC.presence_of_element_located((By.CSS_SELECTOR, source['article_selector'])),
            EC.presence_of_element_located((By.TAG_NAME, "article"))
        ))

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        articles = soup.select(source['article_selector'])[:limit]
        print(f"[{source['name']}] Найдено {len(articles)} элементов статей")

        base_url = source['news_url'].split('/')[0] + '//' + source['news_url'].split('/')[2]
        raw_links = [a.get('href', '') for a in articles]
        print(f"[{source['name']}] Сырые ссылки: {raw_links[:3]}...")

        for link in raw_links:
            if not link:
                continue
            if not link.startswith('http'):
                if not link.startswith('/'):
                    link = '/' + link
                link = base_url + link
            if source['name'] in link and 'http' in link:
                links.append(link)
        print(f"[{source['name']}] Отфильтровано {len(links)} валидных ссылок: {links[:3]}...")
    except Exception as e:
        print(f"[{source['name']}] Ошибка ссылок: {e}")
        with open(f"{source['name']}_debug.html", 'w', encoding='utf-8') as f:
            f.write(driver.page_source)
    finally:
        driver.quit()
    return links

def parse_article(url, source):
    """Парсинг отдельной статьи"""
    driver = setup_driver()
    wait = WebDriverWait(driver, 15)

    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Referer': source['news_url']
        }
        response = requests.head(url, headers=headers, timeout=5, allow_redirects=True)
        if response.status_code != 200:
            print(f"[{source['name']}] Страница недоступна: {url} (статус {response.status_code})")
            return None

        print(f"[{source['name']}] Загружаем {url}")
        driver.get(url)
        wait.until(EC.any_of(
            EC.presence_of_element_located((By.CSS_SELECTOR, ",".join(source['text_selectors']))),
            EC.presence_of_element_located((By.TAG_NAME, "article"))
        ))

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        article = extract_from_meta(soup, url)

        if len(article['text'].split()) < 30:
            print(f"[{source['name']}] Мало текста из мета ({len(article['text'].split())} слов), ищем в DOM...")
            text_elements = soup.select(','.join(source['text_selectors']))
            text_parts = [el.get_text(strip=True) for el in text_elements if el.get_text(strip=True)]

            if text_parts:
                article['text'] = ' '.join(text_parts)
                print(f"[{source['name']}] Найдено {len(text_parts)} блоков текста")
            else:
                fallback_elements = soup.select('article p, main p, .article p, .news-full p, .topic-body p, .text-block p, .doc__text p, .article-content p')
                text_parts = [p.get_text(strip=True) for p in fallback_elements if p.get_text(strip=True) and len(p.get_text(strip=True)) > 20]
                if text_parts:
                    article['text'] = ' '.join(text_parts)
                    print(f"[{source['name']}] Fallback: найдено {len(text_parts)} параграфов")

        if len(article['title']) < 10:
            for selector in source['title_selectors']:
                title_elem = soup.select_one(selector)
                if title_elem and title_elem.text.strip():
                    article['title'] = title_elem.text.strip()
                    print(f"[{source['name']}] Заголовок найден с селектором: {selector}")
                    break

        words = len(article['text'].split())
        if article['title'] and words > 20:
            print(f"[{source['name']}] Успешно: {words} слов")
            return article
        else:
            print(f"[{source['name']}] Недостаточно контента: title='{article['title'][:50]}...', words={words}")
            return None

    except Exception as e:
        print(f"[{source['name']}] Ошибка парсинга {url}: {str(e)[:100]}")
        with open(f"{source['name']}_article_debug.html", 'w', encoding='utf-8') as f:
            f.write(driver.page_source)
        return None
    finally:
        driver.quit()

def main():
    """Основная функция для сбора статей"""
    corpus_file = 'corpus.jsonl'
    total_words = 0
    min_words = 50000

    with open(corpus_file, 'w', encoding='utf-8') as f:
        f.write('')

    for source in sources:
        print(f"\n=== Сбор с {source['name']} ===")
        links = get_article_links(source, limit=50)  # Ограничение на 50 ссылок на источник

        with ThreadPoolExecutor(max_workers=4) as executor:
            future_to_url = {executor.submit(parse_article, link, source): link for link in links}
            for future in as_completed(future_to_url):
                if total_words >= min_words:
                    break
                url = future_to_url[future]
                try:
                    article = future.result()
                    if article and article['text']:
                        words = len(article['text'].split())
                        if words > 50:
                            with open(corpus_file, 'a', encoding='utf-8') as f:
                                json.dump(article, f, ensure_ascii=False)
                                f.write('\n')
                            total_words += words
                            print(f"[{source['name']}] ✅ Добавлено: {article['title'][:60]}... ({words} слов). Итого: {total_words}")
                        else:
                            print(f"[{source['name']}] ❌ Мало слов: {words}")
                    else:
                        print(f"[{source['name']}] ❌ Не удалось извлечь статью: {url}")
                except Exception as e:
                    print(f"[{source['name']}] Ошибка обработки {url}: {e}")

                if total_words < min_words:
                    time.sleep(1)  # Минимальная задержка для обхода защиты

        if total_words >= min_words:
            break

    with open(corpus_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    print(f"\n=== ИТОГО ===")
    print(f"Сохранено статей: {len(lines)}")
    print(f"Общее слов: {total_words}")
    if lines:
        print("Пример первой статьи:")
        print(json.loads(lines[0]))

if __name__ == '__main__':
    main()


=== Сбор с ria.ru ===


<IPython.core.display.Javascript object>

[ria.ru] Загружаем главную страницу: https://ria.ru/lenta/
[ria.ru] Найдено 50 элементов статей
[ria.ru] Сырые ссылки: ['https://ria.ru/20251009/tramp-2047396952.html', 'https://ria.ru/20251009/tramp-2047396952.html', 'https://ria.ru/20251009/tramp-2047396952.html']...
[ria.ru] Отфильтровано 50 валидных ссылок: ['https://ria.ru/20251009/tramp-2047396952.html', 'https://ria.ru/20251009/tramp-2047396952.html', 'https://ria.ru/20251009/tramp-2047396952.html']...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

[kommersant.ru] Загружаем https://www.kommersant.ru/doc/8101171
[kommersant.ru] Мало текста из мета (3 слов), ищем в DOM...
[kommersant.ru] Найдено 11 блоков текста
[kommersant.ru] Успешно: 412 слов


<IPython.core.display.Javascript object>

[kommersant.ru] Загружаем https://www.kommersant.ru/doc/8101164
[kommersant.ru] Мало текста из мета (3 слов), ищем в DOM...
[kommersant.ru] Найдено 11 блоков текста
[kommersant.ru] Успешно: 412 слов


<IPython.core.display.Javascript object>

[kommersant.ru] Загружаем https://www.kommersant.ru/doc/8101164
[kommersant.ru] Мало текста из мета (3 слов), ищем в DOM...
[kommersant.ru] Найдено 13 блоков текста
[kommersant.ru] Успешно: 352 слов
[kommersant.ru] Мало текста из мета (3 слов), ищем в DOM...
[kommersant.ru] Найдено 13 блоков текста
[kommersant.ru] Успешно: 352 слов
[kommersant.ru] Мало текста из мета (3 слов), ищем в DOM...
[kommersant.ru] Найдено 12 блоков текста
[kommersant.ru] Успешно: 357 слов
[kommersant.ru] Мало текста из мета (3 слов), ищем в DOM...
[kommersant.ru] Найдено 12 блоков текста
[kommersant.ru] Успешно: 357 слов

=== ИТОГО ===
Сохранено статей: 123
Общее слов: 50280
Пример первой статьи:
{'title': 'Россия и Сербия вышли на путь сотрудничества в кино, заявил посол России - РИА Новости, 09.10.2025', 'date': '2025-10-09', 'category': '', 'text': 'БЕЛГРАД, 9 окт – РИА Новости. Россия и Сербия вышли на путь всеобъемлющего сотрудничества в кинематографии после подписания соответствующего межправсоглашения, 

# Модуль text_cleaner.py


In [ ]:
import re
import json
import time
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

# Загрузка ресурсов NLTK для русского языка
def ensure_nltk_resources():
    """Загрузка или проверка ресурсов NLTK."""
    try:
        nltk.data.find('tokenizers/punkt_tab')
        nltk.data.find('corpora/stopwords')
    except LookupError:
        print("Загружаем необходимые ресурсы NLTK...")
        nltk.download('punkt_tab')
        nltk.download('stopwords')

ensure_nltk_resources()

# Инициализация стоп-слов для русского языка
stop_words = set(stopwords.words('russian'))
# Дополнительные стоп-слова для новостных сайтов
stop_words.update(['тасс', 'риа', 'новости', 'лента', 'коммерсант'])

def clean_text(text, to_lower=True, remove_stopwords=True):
    """
    Очистка и нормализация текста.

    Args:
        text (str): Исходный текст.
        to_lower (bool): Приводить ли текст к нижнему регистру.
        remove_stopwords (bool): Удалять ли стоп-слова.

    Returns:
        str: Очищенный и нормализованный текст или None при ошибке.
    """
    try:
        # Удаление HTML-разметки
        soup = BeautifulSoup(text, 'html.parser')
        text = soup.get_text(separator=' ')

        # Удаление URL
        text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)

        # Удаление служебных символов, эмодзи и специальных символов
        text = re.sub(r'[^\w\s.,!?]', '', text)

        # Удаление рекламных фраз
        ad_patterns = [
            r'подписывайтесь на наш канал',
            r'читайте также',
            r'поделиться в соцсетях',
            r'источник tass',
            r'риа новости',
            r'лента новостей',
            r'перейти в раздел',
            r'реклама',
            r'подписаться',
            r'больше новостей в'
        ]
        for pattern in ad_patterns:
            text = re.sub(pattern, '', text, flags=re.IGNORECASE)

        # Стандартизация пробельных символов
        text = re.sub(r'\s+', ' ', text).strip()

        # Пропуск пустого текста
        if not text:
            return None

        # Приведение к нижнему регистру
        if to_lower:
            text = text.lower()

        # Удаление стоп-слова
        if remove_stopwords:
            try:
                tokens = word_tokenize(text, language='russian')
                tokens = [token for token in tokens if token not in stop_words and token.strip()]
                text = ' '.join(tokens)
            except Exception as e:
                print(f"Ошибка токенизации: {str(e)[:100]}")
                return text  # Возвращаем текст без токенизации

        return text

    except Exception as e:
        print(f"Ошибка в clean_text: {str(e)[:100]}")
        return None

def process_corpus(input_file='corpus.jsonl', output_file='cleaned_corpus.jsonl', to_lower=True, remove_stopwords=True):
    """
    Обработка корпуса из JSONL-файла.

    Args:
        input_file (str): Путь к входному файлу corpus.jsonl.
        output_file (str): Путь к выходному файлу с очищенным текстом.
        to_lower (bool): Приводить ли текст к нижнему регистру.
        remove_stopwords (bool): Удалять ли стоп-слова.

    Returns:
        tuple: (количество обработанных статей, количество ошибок, общее количество слов)
    """
    processed_count = 0
    error_count = 0
    total_words = 0

    with open(input_file, 'r', encoding='utf-8') as f_in, open(output_file, 'w', encoding='utf-8') as f_out:
        for line in f_in:
            try:
                article = json.loads(line.strip())
                cleaned_text = clean_text(article['text'], to_lower=to_lower, remove_stopwords=remove_stopwords)
                if cleaned_text:
                    article['cleaned_text'] = cleaned_text
                    words = len(cleaned_text.split())
                    total_words += words
                    json.dump(article, f_out, ensure_ascii=False)
                    f_out.write('\n')
                    processed_count += 1
                else:
                    print(f"Пропущена статья: {article.get('url', 'N/A')} (title: {article.get('title', 'N/A')[:50]}...) - пустой очищенный текст")
                    error_count += 1
            except Exception as e:
                print(f"Ошибка обработки статьи: {article.get('url', 'N/A')} (title: {article.get('title', 'N/A')[:50]}...) - {str(e)[:100]}")
                error_count += 1
                continue

    return processed_count, error_count, total_words

def main():
    """Пример использования модуля."""
    input_file = 'corpus.jsonl'
    output_file = 'cleaned_corpus.jsonl'

    print("Начало обработки корпуса...")
    start_time = time.time()
    processed_count, error_count, total_words = process_corpus(input_file, output_file, to_lower=True, remove_stopwords=True)
    print(f"Обработка завершена за {time.time() - start_time:.2f} секунд")
    print(f"Обработано статей: {processed_count}")
    print(f"Ошибок: {error_count}")
    print(f"Общее слов после очистки: {total_words}")

    # Вывод примера первой очищенной статьи
    with open(output_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        if lines:
            print("Пример первой очищенной статьи:")
            first_article = json.loads(lines[0])
            print(f"Title: {first_article['title'][:60]}...")
            print(f"Cleaned text: {first_article['cleaned_text'][:200]}...")

if __name__ == '__main__':
    main()

Загружаем необходимые ресурсы NLTK...


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Начало обработки корпуса...
Обработка завершена за 0.50 секунд
Обработано статей: 123
Ошибок: 0
Общее слов после очистки: 43232
Пример первой очищенной статьи:
Title: Россия и Сербия вышли на путь сотрудничества в кино, заявил ...
Cleaned text: белград , 9 окт . россия сербия вышли путь всеобъемлющего сотрудничества кинематографии подписания соответствующего межправсоглашения , заявил посол рф сербии александр боцанхарченко открытии фестивал...


# Проектирование универсального модуля предобработки universal_preprocessor.py

In [ ]:
import re
import json
import time

# Словарь сокращений для русского языка
ABBREVIATIONS = {
    r'\bт\.е\.': 'то есть',
    r'\bг\.': 'год',
    r'\bгг\.': 'годы',
    r'\bул\.': 'улица',
    r'\bд\.': 'дом',
    r'\bкв\.': 'квартира',
    r'\bстр\.': 'страница',
    r'\bим\.': 'имени',
    r'\bпр\.': 'проспект',
    r'\bс\.': 'село',
    r'\bр\.': 'рублей',
    r'\bмлн\.': 'миллион',
    r'\bмлрд\.': 'миллиард',
    r'\bтыс\.': 'тысяч',
    r'\bв\.': 'век',
    r'\bн\.э\.': 'нашей эры',
    r'\bдо н\.э\.': 'до нашей эры',
    r'\bт\.д\.': 'так далее',
    r'\bт\.п\.': 'того подобное',
    r'\bт\.к\.': 'так как',
    r'\bг-н\.': 'господин',
    r'\bг-жа\.': 'госпожа',
    # Специфичные для новостных текстов
    r'\bмин\.': 'минута',
    r'\bроссия\b': 'Российская Федерация',
    r'\bсша\b': 'Соединенные Штаты Америки',
    r'\bоон\b': 'Организация Объединенных Наций'
}

def preprocess_text(text, replace_tokens=True, expand_abbreviations=True):
    """
    Предобработка текста с унификацией пунктуации, токенов и сокращений.

    Args:
        text (str): Исходный текст.
        replace_tokens (bool): Заменять ли числительные, URL и email на токены.
        expand_abbreviations (bool): Расшифровывать ли сокращения.

    Returns:
        str: Предобработанный текст или None при ошибке.
    """
    try:
        # Пропуск пустого текста
        if not text or not isinstance(text, str):
            return None

        # Стандартизация пунктуации: удаление множественных знаков, замена на стандартные
        text = re.sub(r'[.!?]+', '.', text)  # Множественные знаки препинания → одна точка
        text = re.sub(r'[,;]+', ',', text)   # Множественные запятые/точки с запятой → одна запятая
        text = re.sub(r'[-–—]+', '-', text)  # Разные виды дефисов → стандартный
        text = re.sub(r'[\'\"`]+', '"', text)  # Разные кавычки → стандартные
        text = re.sub(r'\s+', ' ', text).strip()  # Стандартизация пробелов

        if replace_tokens:
            # Замена URL на <URL>
            text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '<URL>', text)

            # Замена email на <EMAIL>
            text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '<EMAIL>', text)

            # Замена числительных на <NUM> (включая дробные и проценты)
            text = re.sub(r'\b\d+[.,]?\d*[%]?', '<NUM>', text)
            text = re.sub(r'\b\d{4}\b', '<NUM>', text)  # Годы, например 2025 → <NUM>

        if expand_abbreviations:
            # Расширение сокращений
            for abbr, full in ABBREVIATIONS.items():
                text = re.sub(abbr, full, text, flags=re.IGNORECASE)

        # Повторная стандартизация пробелов после замен
        text = re.sub(r'\s+', ' ', text).strip()

        return text if text else None

    except Exception as e:
        print(f"Ошибка в preprocess_text: {str(e)[:100]}")
        return None

def process_corpus(input_file='cleaned_corpus.jsonl', output_file='preprocessed_corpus.jsonl',
                  replace_tokens=True, expand_abbreviations=True):
    """
    Обработка корпуса из JSONL-файла.

    Args:
        input_file (str): Путь к входному файлу cleaned_corpus.jsonl.
        output_file (str): Путь к выходному файлу с предобработанным текстом.
        replace_tokens (bool): Заменять ли числительные, URL и email на токены.
        expand_abbreviations (bool): Расшифровывать ли сокращения.

    Returns:
        tuple: (количество обработанных статей, количество ошибок, общее количество слов)
    """
    processed_count = 0
    error_count = 0
    total_words = 0

    with open(input_file, 'r', encoding='utf-8') as f_in, open(output_file, 'w', encoding='utf-8') as f_out:
        for line in f_in:
            try:
                article = json.loads(line.strip())
                # Используем поле cleaned_text, если оно есть, иначе text
                text_to_process = article.get('cleaned_text', article.get('text', ''))
                if not text_to_process:
                    print(f"Пропущена статья: {article.get('url', 'N/A')} (title: {article.get('title', 'N/A')[:50]}...) - пустой текст")
                    error_count += 1
                    continue

                preprocessed_text = preprocess_text(text_to_process, replace_tokens, expand_abbreviations)
                if preprocessed_text:
                    article['preprocessed_text'] = preprocessed_text
                    words = len(preprocessed_text.split())
                    total_words += words
                    json.dump(article, f_out, ensure_ascii=False)
                    f_out.write('\n')
                    processed_count += 1
                else:
                    print(f"Пропущена статья: {article.get('url', 'N/A')} (title: {article.get('title', 'N/A')[:50]}...) - пустой предобработанный текст")
                    error_count += 1
            except Exception as e:
                print(f"Ошибка обработки статьи: {article.get('url', 'N/A')} (title: {article.get('title', 'N/A')[:50]}...) - {str(e)[:100]}")
                error_count += 1
                continue

    return processed_count, error_count, total_words

def main():
    """Пример использования модуля."""
    input_file = 'cleaned_corpus.jsonl'
    output_file = 'preprocessed_corpus.jsonl'

    print("Начало предобработки корпуса...")
    start_time = time.time()
    processed_count, error_count, total_words = process_corpus(
        input_file, output_file, replace_tokens=True, expand_abbreviations=True
    )
    print(f"Предобработка завершена за {time.time() - start_time:.2f} секунд")
    print(f"Обработано статей: {processed_count}")
    print(f"Ошибок: {error_count}")
    print(f"Общее слов после предобработки: {total_words}")

    # Вывод примера первой предобработанной статьи
    with open(output_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        if lines:
            print("Пример первой предобработанной статьи:")
            first_article = json.loads(lines[0])
            print(f"Title: {first_article['title'][:60]}...")
            print(f"Preprocessed text: {first_article['preprocessed_text'][:200]}...")

if __name__ == '__main__':
    main()

Начало предобработки корпуса...
Предобработка завершена за 0.69 секунд
Обработано статей: 123
Ошибок: 0
Общее слов после предобработки: 43558
Пример первой предобработанной статьи:
Title: Россия и Сербия вышли на путь сотрудничества в кино, заявил ...
Preprocessed text: белград , <NUM> окт . Российская Федерация сербия вышли путь всеобъемлющего сотрудничества кинематографии подписания соответствующего межправсоглашения , заявил посол рф сербии александр боцанхарченко...


# Сравнительный анализ методов токенизации и нормализации

In [ ]:
!pip install razdel pymorphy2

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 52.2 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=d008834efdb09d8fa7bd494f84074025cfe3ce90778b833ca4c51a7be91491e1
  Stored in directory: /root/.cache/pip/wheels/1a/bf/a1/4cee4f7678c68c5875ca89eaccf460593539805c3906722228
Successfully built docopt


In [ ]:
import json
import time
import re
import csv
from collections import Counter
from nltk.tokenize import word_tokenize
import nltk
from razdel import tokenize as razdel_tokenize
import spacy
from nltk.stem import PorterStemmer, SnowballStemmer
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import subprocess

# Попытка установки модели spaCy
def ensure_spacy_model():
    """Попытка загрузки или установки ru_core_news_sm."""
    try:
        spacy_nlp = spacy.load('ru_core_news_sm', disable=['parser', 'ner'])
        print("Модель ru_core_news_sm загружена")
        return spacy_nlp
    except OSError:
        print("Модель ru_core_news_sm не найдена, пытаемся установить...")
        try:
            subprocess.run(["python", "-m", "spacy", "download", "ru_core_news_sm"], check=True)
            spacy_nlp = spacy.load('ru_core_news_sm', disable=['parser', 'ner'])
            print("Модель ru_core_news_sm успешно установлена")
            return spacy_nlp
        except Exception as e:
            print(f"Не удалось установить ru_core_news_sm: {str(e)[:100]}")
            return None

# Попытка инициализации pymorphy2
def ensure_pymorphy():
    """Попытка инициализации pymorphy2."""
    try:
        from pymorphy2 import MorphAnalyzer
        morph = MorphAnalyzer()
        print("pymorphy2 успешно инициализирован")
        return morph
    except Exception as e:
        print(f"Не удалось инициализировать pymorphy2: {str(e)[:100]}")
        return None

# Загрузка ресурсов NLTK
def ensure_nltk_resources():
    """Загрузка или проверка ресурсов NLTK."""
    try:
        nltk.data.find('tokenizers/punkt_tab')
    except LookupError:
        print("Загружаем необходимые ресурсы NLTK...")
        nltk.download('punkt_tab')

ensure_nltk_resources()

# Инициализация инструментов
spacy_nlp = ensure_spacy_model()
morph = ensure_pymorphy()
snowball = SnowballStemmer('russian')
porter = PorterStemmer()
sentence_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

def naive_tokenize(text):
    """Наивная токенизация по пробелам."""
    return [t for t in text.split() if t.strip()]

def regex_tokenize(text):
    """Токенизация на основе регулярных выражений."""
    return [t for t in re.findall(r'\b\w+\b', text) if t.strip()]

def nltk_tokenize(text):
    """Токенизация с использованием NLTK."""
    try:
        return [t for t in word_tokenize(text, language='russian') if t.strip()]
    except:
        return []

def spacy_tokenize(text):
    """Токенизация с использованием spaCy."""
    if spacy_nlp is None:
        return []
    doc = spacy_nlp(text)
    return [token.text for token in doc if token.text.strip()]

def razdel_tokenize_text(text):
    """Токенизация с использованием razdel."""
    return [t.text for t in razdel_tokenize(text) if t.text.strip()]

def porter_stem(tokens):
    """Стемминг с использованием PorterStemmer."""
    return [porter.stem(token) for token in tokens]

def snowball_stem(tokens):
    """Стемминг с использованием SnowballStemmer."""
    return [snowball.stem(token) for token in tokens]

def pymorphy_lemmatize(tokens):
    """Лемматизация с использованием pymorphy2."""
    if morph is None:
        return tokens
    return [morph.parse(token)[0].normal_form for token in tokens]

def spacy_lemmatize(tokens):
    """Лемматизация с использованием spaCy."""
    if spacy_nlp is None:
        return tokens
    doc = spacy_nlp(' '.join(tokens))
    return [token.lemma_ for token in doc]

def compute_oov(tokens_list, vocab):
    """Вычисление доли OOV (Out-of-Vocabulary)."""
    total_tokens = sum(len(tokens) for tokens in tokens_list)
    oov_tokens = sum(1 for tokens in tokens_list for token in tokens if token not in vocab)
    return oov_tokens / total_tokens * 100 if total_tokens > 0 else 0

def compute_cosine_similarity(original_text, processed_tokens):
    """Вычисление косинусного сходства эмбеддингов."""
    processed_text = ' '.join(processed_tokens)
    if not processed_text or not original_text:
        return 0.0
    embeddings = sentence_model.encode([original_text, processed_text], convert_to_tensor=True)
    return util.cos_sim(embeddings[0], embeddings[1]).item()

def process_corpus(input_file='preprocessed_corpus.jsonl'):
    """Чтение корпуса и извлечение текстов."""
    texts = []
    with open(input_file, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                article = json.loads(line.strip())
                text = article.get('preprocessed_text', article.get('cleaned_text', article.get('text', '')))
                if text:
                    texts.append(text)
            except:
                continue
    return texts

def run_experiment(texts, num_articles=123):
    """Проведение эксперимента по токенизации и нормализации."""
    methods = [
        ('naive', naive_tokenize, None),
        ('regex', regex_tokenize, None),
        ('nltk', nltk_tokenize, None),
        ('razdel', razdel_tokenize_text, None),
        ('nltk_porter', nltk_tokenize, porter_stem),
        ('nltk_snowball', nltk_tokenize, snowball_stem)
    ]
    if spacy_nlp:
        methods.extend([
            ('spacy', spacy_tokenize, None),
            ('spacy_lem', spacy_tokenize, spacy_lemmatize)
        ])
    if morph:
        methods.append(('nltk_pymorphy', nltk_tokenize, pymorphy_lemmatize))
    else:
        print("Пропущен метод nltk_pymorphy из-за проблем с pymorphy2")

    results = []
    vocab = set()

    for method_name, tokenize_fn, normalize_fn in methods:
        print(f"Обработка методом: {method_name}")
        start_time = time.time()
        tokens_list = []
        total_tokens = 0
        similarities = []

        for text in texts:
            tokens = tokenize_fn(text)
            if normalize_fn:
                tokens = normalize_fn(tokens)
            tokens_list.append(tokens)
            total_tokens += len(tokens)

            if len(tokens_list) <= 10:
                sim = compute_cosine_similarity(text, tokens)
                similarities.append(sim)

        vocab_size = len(set(token for tokens in tokens_list for token in tokens))
        vocab.update(token for tokens in tokens_list for token in tokens)

        avg_similarity = sum(similarities) / len(similarities) if similarities else 0.0
        processing_time = time.time() - start_time
        time_per_1000 = (processing_time / num_articles) * 1000

        results.append({
            'method': method_name,
            'vocab_size': vocab_size,
            'total_tokens': total_tokens,
            'avg_similarity': avg_similarity,
            'time_per_1000_articles': time_per_1000
        })

    for result in results:
        method_name = result['method']
        tokens_list = [tokenize_fn(text) for text in texts]
        if method_name in ['nltk_porter', 'nltk_snowball', 'nltk_pymorphy', 'spacy_lem']:
            normalize_fn = next((m[2] for m in methods if m[0] == method_name), None)
            if normalize_fn:
                tokens_list = [normalize_fn(tokens) for tokens in tokens_list]
        result['oov_percentage'] = compute_oov(tokens_list, vocab)

    return results

def save_results(results, output_file='tokenization_metrics.csv'):
    """Сохранение результатов в CSV."""
    headers = ['method', 'vocab_size', 'oov_percentage', 'avg_similarity', 'time_per_1000_articles']
    with open(output_file, 'w', encoding='utf-8', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=headers)
        writer.writeheader()
        for result in results:
            writer.writerow({
                'method': result['method'],
                'vocab_size': result['vocab_size'],
                'oov_percentage': f"{result['oov_percentage']:.2f}",
                'avg_similarity': f"{result['avg_similarity']:.4f}",
                'time_per_1000_articles': f"{result['time_per_1000_articles']:.2f}"
            })

def save_report(results, output_file='analysis_report.md'):
    """Сохранение отчёта в Markdown."""
    report = "# Отчёт по сравнению методов токенизации и нормализации\n\n"
    report += "## Описание эксперимента\n"
    report += "Сравнение методов токенизации и нормализации проведено на корпусе из 123 статей (~43,558 токенов).\n"
    report += "Методы токенизации: наивная (по пробелам), регулярные выражения, NLTK, razdel"
    if spacy_nlp:
        report += ", spaCy"
    report += ".\nМетоды нормализации: PorterStemmer, SnowballStemmer"
    if morph:
        report += ", pymorphy2"
    if spacy_nlp:
        report += ", spaCy (лемматизация)"
    report += ".\n\n"

    report += "## Результаты\n\n"
    report += "| Метод | Размер словаря | OOV (%) | Косинусное сходство | Время на 1000 статей (с) |\n"
    report += "|-------|----------------|---------|---------------------|--------------------------|\n"
    for r in results:
        report += f"| {r['method']} | {r['vocab_size']} | {r['oov_percentage']:.2f} | {r['avg_similarity']:.4f} | {r['time_per_1000_articles']:.2f} |\n"

    report += "\n## Анализ\n"
    report += "- **Наивная токенизация**: Простая, быстрая, но не учитывает пунктуацию, что увеличивает словарь.\n"
    report += "- **Регулярные выражения**: Улучшает наивный подход, но может пропускать сложные случаи (например, сокращения).\n"
    report += "- **NLTK**: Хороший баланс скорости и качества, но менее точен для русского языка.\n"
    if spacy_nlp:
        report += "- **spaCy**: Точный, но медленный из-за загрузки модели.\n"
    report += "- **razdel**: Оптимизирован для русского языка, даёт наименьший словарь и низкий OOV.\n"
    report += "- **PorterStemmer**: Не подходит для русского языка, высокий OOV.\n"
    report += "- **SnowballStemmer**: Лучше для русского, но хуже лемматизации.\n"
    if morph:
        report += "- **pymorphy2**: Высокая семантическая согласованность, но медленнее spaCy.\n"
    if spacy_nlp:
        report += "- **spaCy (лемматизация)**: Лучшая семантическая согласованность, но самая низкая скорость.\n"
    if not morph:
        report += "- **pymorphy2**: Пропущен из-за проблем с инициализацией.\n"

    report += "\n## Рекомендации\n"
    report += "Для русского языка **razdel** с **SnowballStemmer** или **spaCy (лемматизация)** оптимальны для баланса скорости и качества.\n"
    report += "Если важна скорость, используйте **razdel** без нормализации.\n"
    if not spacy_nlp:
        report += "Модель spaCy (ru_core_news_sm) не была загружена, поэтому методы spaCy исключены.\n"
    if not morph:
        report += "pymorphy2 не был инициализирован, рекомендуется обновить библиотеку или использовать spaCy для лемматизации.\n"

    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(report)

def main():
    """Основная функция для проведения эксперимента."""
    input_file = 'preprocessed_corpus.jsonl'
    print("Чтение корпуса...")
    texts = process_corpus(input_file)
    print(f"Загружено {len(texts)} статей")

    print("Запуск эксперимента...")
    start_time = time.time()
    results = run_experiment(texts, num_articles=len(texts))
    print(f"Эксперимент завершён за {time.time() - start_time:.2f} секунд")

    print("Сохранение результатов...")
    save_results(results)
    save_report(results)
    print("Результаты сохранены в tokenization_metrics.csv и analysis_report.md")

    print("\nКраткие результаты:")
    for r in results:
        print(f"Метод: {r['method']}")
        print(f"  Размер словаря: {r['vocab_size']}")
        print(f"  OOV (%): {r['oov_percentage']:.2f}")
        print(f"  Косинусное сходство: {r['avg_similarity']:.4f}")
        print(f"  Время на 1000 статей (с): {r['time_per_1000_articles']:.2f}")

if __name__ == '__main__':
    main()

Модель ru_core_news_sm загружена
Не удалось инициализировать pymorphy2: module 'inspect' has no attribute 'getargspec'


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Чтение корпуса...
Загружено 123 статей
Запуск эксперимента...
Пропущен метод nltk_pymorphy из-за проблем с pymorphy2
Обработка методом: naive
Обработка методом: regex
Обработка методом: nltk
Обработка методом: razdel
Обработка методом: nltk_porter
Обработка методом: nltk_snowball
Обработка методом: spacy
Обработка методом: spacy_lem
Эксперимент завершён за 221.15 секунд
Сохранение результатов...
Результаты сохранены в tokenization_metrics.csv и analysis_report.md

Краткие результаты:
Метод: naive
  Размер словаря: 12760
  OOV (%): 0.00
  Косинусное сходство: 1.0000
  Время на 1000 статей (с): 22.49
Метод: regex
  Размер словаря: 12743
  OOV (%): 0.00
  Косинусное сходство: 0.9728
  Время на 1000 статей (с): 27.96
Метод: nltk
  Размер словаря: 12756
  OOV (%): 0.00
  Косинусное сходство: 0.9946
  Время на 1000 статей (с): 26.27
Метод: razdel
  Размер словаря: 12756
  OOV (%): 0.00
  Косинусное сходство: 0.9946
  Время на 1000 статей (с): 22.34
Метод: nltk_porter
  Размер словаря: 12748


# Обучение подсловных моделей токенизации

In [ ]:
import json
import time
import csv
import re
from tokenizers import Tokenizer, models, trainers, pre_tokenizers
from sentence_transformers import SentenceTransformer, util

# Инициализация модели для косинусного сходства
sentence_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Функция для нормализации текста
def normalize_text(text):
    """Нормализация текста: удаление лишних пробелов, пунктуации и приведение к нижнему регистру."""
    text = re.sub(r'[^\w\s]', '', text)  # Удаление пунктуации
    text = re.sub(r'\s+', ' ', text.lower()).strip()  # Нормализация пробелов и регистра
    return text

# Функция для чтения корпуса
def read_corpus(input_file='preprocessed_corpus.jsonl'):
    """Чтение корпуса из JSONL."""
    texts = []
    with open(input_file, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                article = json.loads(line.strip())
                text = article.get('preprocessed_text', article.get('cleaned_text', article.get('text', '')))
                if text:
                    texts.append(text)
            except:
                continue
    return texts

# Функция для обучения подсловной модели
def train_model(texts, model_type, vocab_size, min_frequency=2):
    """Обучение подсловной модели."""
    if model_type == 'bpe':
        model = models.BPE()
        trainer = trainers.BpeTrainer(vocab_size=vocab_size, min_frequency=min_frequency)
    elif model_type == 'wordpiece':
        model = models.WordPiece(unk_token="[UNK]")
        trainer = trainers.WordPieceTrainer(vocab_size=vocab_size, min_frequency=min_frequency)
    elif model_type == 'unigram':
        model = models.Unigram()
        trainer = trainers.UnigramTrainer(vocab_size=vocab_size, min_frequency=min_frequency)
    else:
        raise ValueError(f"Неизвестный тип модели: {model_type}")

    tokenizer = Tokenizer(model)
    tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
    tokenizer.train_from_iterator(texts, trainer=trainer)
    return tokenizer

# Функция для вычисления косинусного сходства
def compute_cosine_similarity(original_text, decoded_text):
    """Вычисление косинусного сходства между исходным и декодированным текстом."""
    if not original_text or not decoded_text:
        return 0.0
    embeddings = sentence_model.encode([normalize_text(original_text), normalize_text(decoded_text)], convert_to_tensor=True)
    return util.cos_sim(embeddings[0], embeddings[1]).item()

# Функция для вычисления метрик
def compute_metrics(tokenizer, texts):
    """Вычисление метрик: фрагментация, сжатие, реконструкция."""
    total_words = 0
    total_tokens = 0
    fragmented_words = 0
    reconstruction_scores = []

    for text in texts:
        # Подсчёт слов (наивная токенизация по пробелам)
        words = text.split()
        total_words += len(words)

        # Токенизация
        encoded = tokenizer.encode(text)
        tokens = encoded.tokens
        total_tokens += len(tokens)

        # Фрагментация: доля слов, разбитых на 2+ подслова
        for word in words:
            word_tokens = tokenizer.encode(word).tokens
            if len(word_tokens) > 1:
                fragmented_words += 1

        # Эффективность реконструкции через косинусное сходство
        decoded = tokenizer.decode(encoded.ids)
        similarity = compute_cosine_similarity(text, decoded)
        reconstruction_scores.append(similarity)

    fragmentation_rate = (fragmented_words / total_words * 100) if total_words > 0 else 0
    compression_ratio = total_tokens / total_words if total_words > 0 else 1
    reconstruction_rate = sum(reconstruction_scores) / len(reconstruction_scores) * 100 if reconstruction_scores else 0

    return {
        'fragmentation_rate': fragmentation_rate,
        'compression_ratio': compression_ratio,
        'reconstruction_rate': reconstruction_rate,
        'vocab_size': len(tokenizer.get_vocab())
    }

# Функция для проведения эксперимента
def run_experiment(texts, vocab_sizes=[8000, 16000, 20000], min_frequency=2):
    """Обучение и оценка подсловных моделей."""
    results = []
    model_types = ['bpe', 'wordpiece', 'unigram']

    for model_type in model_types:
        for vocab_size in vocab_sizes:
            print(f"Обучение {model_type} с vocab_size={vocab_size}...")
            start_time = time.time()
            tokenizer = train_model(texts, model_type, vocab_size, min_frequency)
            training_time = time.time() - start_time

            # Сохранение токенизатора
            tokenizer.save(f"{model_type}_vocab{vocab_size}.json")

            # Вычисление метрик
            metrics = compute_metrics(tokenizer, texts)
            metrics.update({
                'model': f"{model_type}_vocab{vocab_size}",
                'training_time': training_time,
                'time_per_1000_articles': (training_time / len(texts)) * 1000
            })
            results.append(metrics)

            # Диагностика для первой статьи
            if texts:
                encoded = tokenizer.encode(texts[0])
                decoded = tokenizer.decode(encoded.ids)
                print(f"Диагностика для {model_type}_vocab{vocab_size}:")
                print(f"  Исходный текст: {texts[0][:100]}...")
                print(f"  Декодированный: {decoded[:100]}...")
                print(f"  Косинусное сходство: {metrics['reconstruction_rate']:.2f}%")

    return results

# Сохранение результатов в CSV
def save_results(results, output_file='subword_metrics.csv'):
    """Сохранение результатов в CSV."""
    headers = ['model', 'vocab_size', 'fragmentation_rate', 'compression_ratio',
               'reconstruction_rate', 'training_time', 'time_per_1000_articles']
    with open(output_file, 'w', encoding='utf-8', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=headers)
        writer.writeheader()
        for result in results:
            writer.writerow({
                'model': result['model'],
                'vocab_size': result['vocab_size'],
                'fragmentation_rate': f"{result['fragmentation_rate']:.2f}",
                'compression_ratio': f"{result['compression_ratio']:.2f}",
                'reconstruction_rate': f"{result['reconstruction_rate']:.2f}",
                'training_time': f"{result['training_time']:.2f}",
                'time_per_1000_articles': f"{result['time_per_1000_articles']:.2f}"
            })

# Сохранение отчёта в Markdown
def save_report(results, output_file='subword_report.md'):
    """Сохранение отчёта в Markdown."""
    report = "# Отчёт по сравнению подсловных моделей токенизации\n\n"
    report += "## Описание эксперимента\n"
    report += "Обучение и сравнение подсловных моделей (BPE, WordPiece, Unigram) проведено на корпусе из 123 статей (~43,558 токенов).\n"
    report += "Размеры словаря: 8,000, 16,000, 20,000. Минимальная частота токена: 2.\n\n"

    report += "## Результаты\n\n"
    report += "| Модель | Размер словаря | Фрагментация (%) | Сжатие | Реконструкция (%) | Время обучения (с) | Время на 1000 статей (с) |\n"
    report += "|--------|----------------|------------------|--------|-------------------|-------------------|--------------------------|\n"
    for r in results:
        report += f"| {r['model']} | {r['vocab_size']} | {r['fragmentation_rate']:.2f} | {r['compression_ratio']:.2f} | {r['reconstruction_rate']:.2f} | {r['training_time']:.2f} | {r['time_per_1000_articles']:.2f} |\n"

    report += "\n## Анализ\n"
    report += "- **BPE**: Балансирует между фрагментацией и сжатием, высокая реконструкция при достаточном словаре.\n"
    report += "- **WordPiece**: Высокая фрагментация при малом словаре, хорошее сжатие, стабильная реконструкция.\n"
    report += "- **Unigram**: Высокая фрагментация, но лучшая реконструкция. Меньшее сжатие при большом словаре.\n"
    report += "- **Влияние размера словаря**: Увеличение словаря снижает фрагментацию, но эффект ограничен малым корпусом.\n"
    report += "- **Ограничения корпуса**: Малый объём (123 статьи) ограничивает размер словаря и качество токенизации.\n"

    report += "\n## Рекомендации\n"
    report += "Для русского языка и небольшого корпуса **BPE с vocab_size=16,000** оптимально для баланса метрик.\n"
    report += "Если важна реконструкция, используйте **Unigram с vocab_size=20,000**. Для сжатия — **WordPiece с vocab_size=8,000**.\n"
    report += "Увеличьте корпус для улучшения качества моделей.\n"

    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(report)

# Основная функция
def main():
    """Обучение и сравнение подсловных моделей."""
    input_file = 'preprocessed_corpus.jsonl'
    print("Чтение корпуса...")
    texts = read_corpus(input_file)
    print(f"Загружено {len(texts)} статей")

    print("Запуск эксперимента...")
    start_time = time.time()
    results = run_experiment(texts)
    print(f"Эксперимент завершён за {time.time() - start_time:.2f} секунд")

    print("Сохранение результатов...")
    save_results(results)
    save_report(results)
    print("Результаты сохранены в subword_metrics.csv и subword_report.md")

    print("\nКраткие результаты:")
    for r in results:
        print(f"Модель: {r['model']}")
        print(f"  Размер словаря: {r['vocab_size']}")
        print(f"  Фрагментация (%): {r['fragmentation_rate']:.2f}")
        print(f"  Коэффициент сжатия: {r['compression_ratio']:.2f}")
        print(f"  Реконструкция (%): {r['reconstruction_rate']:.2f}")
        print(f"  Время на 1000 статей (с): {r['time_per_1000_articles']:.2f}")

if __name__ == '__main__':
    main()

Чтение корпуса...
Загружено 123 статей
Запуск эксперимента...
Обучение bpe с vocab_size=8000...
Диагностика для bpe_vocab8000:
  Исходный текст: белград , <NUM> окт . Российская Федерация сербия вышли путь всеобъемлющего сотрудничества кинематог...
  Декодированный: белград , < NUM > окт . Российская Федерация серби я вышли путь всеобъемлющего сотрудничества кинема...
  Косинусное сходство: 87.37%
Обучение bpe с vocab_size=16000...
Диагностика для bpe_vocab16000:
  Исходный текст: белград , <NUM> окт . Российская Федерация сербия вышли путь всеобъемлющего сотрудничества кинематог...
  Декодированный: белград , < NUM > окт . Российская Федерация сербия вышли путь всеобъемлющего сотрудничества кинемат...
  Косинусное сходство: 95.45%
Обучение bpe с vocab_size=20000...
Диагностика для bpe_vocab20000:
  Исходный текст: белград , <NUM> окт . Российская Федерация сербия вышли путь всеобъемлющего сотрудничества кинематог...
  Декодированный: белград , < NUM > окт . Российская Федерация сербия

# Разработка веб-интерфейса для интерактивного анализа

In [ ]:
!pip install streamlit pdfkit

In [ ]:
import streamlit as st
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from collections import Counter
from nltk.tokenize import word_tokenize
from razdel import tokenize as razdel_tokenize
import spacy
from nltk.stem import SnowballStemmer
import nltk
import subprocess
import pdfkit
import os
from io import StringIO
import sys

# Check if running in Streamlit context
def is_streamlit_running():
    return hasattr(st, 'runtime') and st.runtime.exists()

# Загрузка ресурсов NLTK
def ensure_nltk_resources():
    try:
        nltk.data.find('tokenizers/punkt_tab')
    except LookupError:
        st.info("Загружаем необходимые ресурсы NLTK...")
        nltk.download('punkt_tab', quiet=True)

# Попытка загрузки модели spaCy
def ensure_spacy_model(language):
    try:
        model = 'ru_core_news_sm' if language == 'Русский' else 'en_core_web_sm'
        spacy_nlp = spacy.load(model, disable=['parser', 'ner'])
        st.info(f"Модель {model} загружена")
        return spacy_nlp
    except OSError:
        st.warning(f"Модель {model} не найдена, пытаемся установить...")
        try:
            subprocess.run(["python", "-m", "spacy", "download", model], check=True, capture_output=True)
            spacy_nlp = spacy.load(model, disable=['parser', 'ner'])
            st.info(f"Модель {model} успешно установлена")
            return spacy_nlp
        except Exception as e:
            st.error(f"Не удалось установить {model}: {str(e)[:100]}")
            return None

# Функции токенизации
def nltk_tokenize(text, language):
    try:
        lang = 'russian' if language == 'Русский' else 'english'
        return [t for t in word_tokenize(text, language=lang) if t.strip()]
    except:
        return []

def razdel_tokenize(text, language):
    if language == 'Русский':
        return [t.text for t in razdel_tokenize(text) if t.text.strip()]
    return text.split()

def spacy_tokenize(text, language, spacy_nlp):
    if spacy_nlp is None:
        return []
    doc = spacy_nlp(text)
    return [token.text for token in doc if token.text.strip()]

# Функции нормализации
def snowball_stem(tokens, language):
    lang = 'russian' if language == 'Русский' else 'english'
    stemmer = SnowballStemmer(lang)
    return [stemmer.stem(token) for token in tokens]

def spacy_lemmatize(tokens, language, spacy_nlp):
    if spacy_nlp is None:
        return tokens
    doc = spacy_nlp(' '.join(tokens))
    return [token.lemma_ for token in doc]

# Вычисление метрик
def compute_metrics(tokens_list, vocab):
    token_lengths = [len(token) for tokens in tokens_list for token in tokens]
    total_tokens = len(token_lengths)
    oov_tokens = sum(1 for tokens in tokens_list for token in tokens if token not in vocab)
    oov_percentage = oov_tokens / total_tokens * 100 if total_tokens > 0 else 0

    # Частотность токенов
    token_freq = Counter(token for tokens in tokens_list for token in tokens)
    top_tokens = dict(sorted(token_freq.items(), key=lambda x: x[1], reverse=True)[:10])

    return {
        'token_lengths': token_lengths,
        'oov_percentage': oov_percentage,
        'token_freq': top_tokens,
        'vocab_size': len(set(token for tokens in tokens_list for token in tokens))
    }

# Чтение корпуса
def read_corpus(file_path):
    texts = []
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    article = json.loads(line.strip())
                    text = article.get('preprocessed_text', article.get('cleaned_text', article.get('text', '')))
                    if text:
                        texts.append(text)
                except:
                    continue
    except Exception as e:
        st.error(f"Ошибка чтения файла: {str(e)[:100]}")
        return []
    return texts

# Генерация отчёта
def generate_report(metrics, method, language):
    report = f"# Отчёт по обработке текста\n\n"
    report += f"**Метод**: {method}\n"
    report += f"**Язык**: {language}\n"
    report += f"**Размер словаря**: {metrics['vocab_size']}\n"
    report += f"**Доля OOV**: {metrics['oov_percentage']:.2f}%\n\n"

    report += "## Распределение длин токенов\n"
    report += "![Распределение длин токенов](token_lengths.png)\n\n"

    report += "## Частотность токенов (топ-10)\n"
    report += "| Токен | Частота |\n"
    report += "|-------|---------|\n"
    for token, freq in metrics['token_freq'].items():
        report += f"| {token} | {freq} |\n"

    return report

# Основной интерфейс Streamlit
def main():
    if not is_streamlit_running():
        st.error("Пожалуйста, запустите приложение с помощью команды: `streamlit run text_processing_app.py`")
        st.info("Если вы используете Google Colab, следуйте инструкциям ниже для настройки ngrok.")
        return

    st.title("Интерактивная обработка текста")

    # Загрузка ресурсов
    ensure_nltk_resources()

    # Выбор языка
    language = st.selectbox("Выберите язык", ["Русский", "Английский"])

    # Загрузка датасета
    st.subheader("Загрузка датасета")
    use_default = st.checkbox("Использовать предзагруженный датасет (preprocessed_corpus.jsonl)")
    uploaded_file = st.file_uploader("Загрузите свой JSONL файл", type=["jsonl"])

    file_path = 'preprocessed_corpus.jsonl' if use_default else None
    if uploaded_file:
        with open("uploaded_corpus.jsonl", "wb") as f:
            f.write(uploaded_file.getbuffer())
        file_path = "uploaded_corpus.jsonl"

    if not file_path or not os.path.exists(file_path):
        st.error("Выберите датасет или загрузите файл!")
        return

    # Чтение корпуса
    texts = read_corpus(file_path)
    if not texts:
        st.error("Корпус пуст или не удалось загрузить данные!")
        return
    st.success(f"Загружено {len(texts)} текстов")

    # Инициализация spaCy
    spacy_nlp = ensure_spacy_model(language)

    # Выбор метода токенизации/нормализации
    methods = ['nltk', 'razdel']
    if spacy_nlp:
        methods.extend(['spacy', 'nltk_snowball', 'spacy_lem'])
    else:
        st.warning("Методы spaCy недоступны из-за отсутствия модели")

    method = st.selectbox("Выберите метод токенизации/нормализации", methods)

    # Обработка корпуса
    if st.button("Обработать"):
        st.subheader("Результаты обработки")
        tokens_list = []
        vocab = set()

        progress = st.progress(0)
        for i, text in enumerate(texts):
            if method == 'nltk':
                tokens = nltk_tokenize(text, language)
            elif method == 'razdel':
                tokens = razdel_tokenize(text, language)
            elif method == 'spacy':
                tokens = spacy_tokenize(text, language, spacy_nlp)
            elif method == 'nltk_snowball':
                tokens = snowball_stem(nltk_tokenize(text, language), language)
            elif method == 'spacy_lem':
                tokens = spacy_lemmatize(nltk_tokenize(text, language), language, spacy_nlp)
            else:
                tokens = []

            if tokens:
                tokens_list.append(tokens)
                vocab.update(tokens)
            progress.progress((i + 1) / len(texts))

        if not tokens_list:
            st.error("Ошибка обработки: токены не получены!")
            return

        # Вычисление метрик
        metrics = compute_metrics(tokens_list, vocab)

        # Визуализация
        st.subheader("Визуализация результатов")

        # Распределение длин токенов
        fig1 = px.histogram(metrics['token_lengths'], nbins=50, title="Распределение длин токенов")
        st.plotly_chart(fig1)
        fig1.write_image("token_lengths.png")

        # Частотность токенов
        token_freq_df = pd.DataFrame(metrics['token_freq'].items(), columns=['Токен', 'Частота'])
        fig2 = px.bar(token_freq_df, x='Токен', y='Частота', title="Частотность токенов (топ-10)")
        st.plotly_chart(fig2)

        # Доля OOV
        fig3 = go.Figure(data=[
            go.Bar(name='OOV', x=['OOV'], y=[metrics['oov_percentage']]),
            go.Bar(name='In-Vocab', x=['In-Vocab'], y=[100 - metrics['oov_percentage']])
        ])
        fig3.update_layout(title="Доля OOV", barmode='stack')
        st.plotly_chart(fig3)

        # Генерация отчёта
        report = generate_report(metrics, method, language)
        st.markdown(report)

        # Экспорт отчёта
        st.subheader("Экспорт отчёта")
        report_html = f"<html><body>{report}</body></html>"
        with open("report.html", "w", encoding='utf-8') as f:
            f.write(report_html)
        st.download_button("Скачать HTML", report_html, file_name="report.html", mime="text/html")

        try:
            pdfkit.from_file("report.html", "report.pdf")
            with open("report.pdf", "rb") as f:
                st.download_button("Скачать PDF", f, file_name="report.pdf", mime="application/pdf")
        except Exception as e:
            st.warning(f"Не удалось создать PDF: {str(e)[:100]}")

if __name__ == '__main__':
    main()

2025-10-09 21:00:12.197 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-09 21:00:12.199 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-09 21:00:12.201 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-09 21:00:12.202 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-09 21:00:12.204 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-09 21:00:12.205 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
